In [25]:
import pandas as pd
import tweepy
import time
import numpy as np
import matplotlib.pyplot as plt
from tweepy import Stream
from tweepy.streaming import StreamListener
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [26]:
consumer_key  = 'd9Ksoz6Wb1jD0mqbW8rjaSNb7'
consumer_secret = 'pHXnVSJeLbOxaYlbOR7BWFdDNhZSF6IzegZV87qUSUqy6Qe8qG'
access_token = '3648603434-dGRu1nHet22tdoYeqaAGoN8MyZrNw9oXZQvGZUD'
access_token_secret = 'PZ8pcQBCb5zVPLRQNVQZc3Yzi0rz1wPef6O7RO7gzcvOf'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [27]:
api

In [28]:
ls

Fri_1.txt  friend20.csv  spam12.txt


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
cd drive/MyDrive/

[Errno 2] No such file or directory: 'drive/MyDrive/'
/content/drive/MyDrive/NoSqlJ


In [31]:
mkdir NoSqlJ

In [32]:
cd NoSqlJ/

/content/drive/My Drive/NoSqlJ/NoSqlJ


In [34]:
cat > Fri_1.txt

^C


In [35]:
ls

Fri_1.txt


In [38]:
# printing all the friends names of the user 
print('Name of the Friends of user')
friends = []
for friend in tweepy.Cursor(api.friends, screen_name = 'Rajinikanth').items(30):
    try:
                friends.append(friend.screen_name)
                print(friend.screen_name)
                time.sleep()
    except Exception as e:
                pass
with open("Fri_1.txt", "w") as f:
     for item in friends:
        f.write("%s\n" % item)

Name of the Friends of user
anirudhofficial
arrahman
dhanushkraja
soundaryaarajni
ash_r_dhanush
SrBachchan
PMOIndia
narendramodi
firstpost
bsindia
airnewsalerts
EconomicTimes
ABPNews
BBCHindi
aajtak
ZeeNewsEnglish
TimesNow
htTweets
IndiaToday
IndianExpress
ndtv
the_hindu
timesofindia
CNNnews18


In [39]:
Total_Data = []
fo = open("Fri_1.txt", "r")
f = fo.readlines()
fo.close()
dataset = map(lambda s: s.strip(),f)
try:
    for datavar in dataset:
        data = api.get_user(datavar)
        counter = 0
        for status in tweepy.Cursor(api.user_timeline, id = datavar).items(30):
            try:
                counter= counter+1
                Total_Data.append(status)
                time.sleep()
            except Exception as e:
                pass
except Exception as e:
    pass
print(len(Total_Data))

720


In [40]:
import urllib.parse
import pandas as pd

def process_http(string):
    url_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if s and n:
            url_count += 1
    return url_count

def process_hashtag(string):
    hashtag_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '#':
            hashtag_count += 1
    return hashtag_count

def process_mention(string):
    mention_count=0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '@':
            mention_count += 1
    return mention_count
def process_data(Total_Data):
    TwittID = [tweet.id for tweet in Total_Data]
    # Making the dataset in pandas frame
    Data = pd.DataFrame(TwittID, columns = ['TwittID'])
    # processing the data in Tweet level
    
    Data["TextData"] = [tweet.text for tweet in Total_Data]
    Data["TweetCreatedAt"] = [tweet.created_at for tweet in Total_Data]
    Data["RetweetCount"] = [tweet.retweet_count for tweet in Total_Data]
    Data["TweetFavouriteCount"] = [tweet.favorite_count for tweet in Total_Data]
    Data["TweetSource"] = [tweet.source for tweet in Total_Data]
    
    # processing the data in User Graph level
    
    Data["UserID"] = [tweet.author.id for tweet in Total_Data]
    Data["UserScreenName"] = [tweet.author.screen_name for tweet in Total_Data]
    Data["UserName"] = [tweet.author.name for tweet in Total_Data]
    Data["UserCreatedAt"] = [tweet.author.created_at for tweet in Total_Data]
    Data["UserDescription"] = [tweet.author.description for tweet in Total_Data]
    Data["UserDescriptionLength"] = [len(tweet.author.description) for tweet in Total_Data]
    Data["UserFollowersCount"] = [tweet.author.followers_count for tweet in Total_Data]
    Data["UserFriendsCount"] = [tweet.author.friends_count for tweet in Total_Data]
    Data["UserLocation"] = [tweet.author.location for tweet in Total_Data]
    
    # Data["url"] = [tweet.author.url for in Total_Data]
    # Data["User_mention"] = [user_mentions.author.screen_name for tweet in Total_Data]
    # Data["HashTag"] = [hashtag.text for tweet in Total_Data]
    
    Data["HttpCount"] = [process_http(tweet.text) for tweet in Total_Data]
    Data["HashtagCount"] = [process_hashtag(tweet.text) for tweet in Total_Data]
    Data["MentionCount"] = [process_mention(tweet.text) for tweet in Total_Data]
    Data["TweetCount"] = [tweet.author.statuses_count for tweet in Total_Data]
    return Data
Data = process_data(Total_Data)
Data.shape

(720, 19)

In [41]:
Data.head()

,TwittID,TextData,TweetCreatedAt,RetweetCount,TweetFavouriteCount,TweetSource,UserID,UserScreenName,UserName,UserCreatedAt,UserDescription,UserDescriptionLength,UserFollowersCount,UserFriendsCount,UserLocation,HttpCount,HashtagCount,MentionCount,TweetCount
0,1399625976800153600,RT @karthiksubbaraj: Here's #JagameThandhiramT...,2021-06-01 07:16:51,3227,0,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,1,2,4,12891
1,1399625927089217536,RT @dhanushkraja: What would have been a great...,2021-06-01 07:16:39,7527,0,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,0,0,1,12891
2,1395367947552522241,This track and it’s video are 🔥 \nAwesome work...,2021-05-20 13:16:58,302,3645,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,1,1,3,12891
3,1395030557092769794,#URGENT 🆘\nEmergency need of 72 vials of #Amph...,2021-05-19 14:56:17,925,2715,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,1,2,0,12891
4,1392771872056479748,RT @soundaryaarajni: Our Thalaivar gets his va...,2021-05-13 09:21:05,4565,0,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,0,2,1,12891


In [42]:
import sys
# Saving data with item space separating
Data.to_csv('RajiniFriends.csv', sep=',' , encoding='utf8')

## Spam Data Generate

In [48]:
friends = []
class listener(StreamListener):
    def on_data(self, data):
        try:
            tweet = data.split(',"screen_name":"')[1].split('","location')[0]
            print(tweet)
            friends.append(tweet)
            return True
        except BaseException as e:
            print('failed on data' + str(e))
            time.sleep(5)
    def on_error(self, status):
        print(status)

twitterStream = Stream(auth, listener())
try:
    for x in range(1,10):
        twitterStream.filter(track=["cougar"])
except KeyboardInterrupt:
    print("Key board interuption")
with open("SpamData.txt", "w") as f:
     for item in friends:
        f.write("%s\n" % item)

g_mjo1
kannibal31
naughtysussex
YouAssTube
coolpornoo
raaaayvenn
PhenomPro1
Key board interuption


In [49]:
Total_Data = []
fo = open("SpamData.txt", "r")
f = fo.readlines()
fo.close()
dataset = map(lambda s: s.strip(),f)
try:
    for datavar in dataset:
        data = api.get_user(datavar)
        counter = 0
        for status in tweepy.Cursor(api.user_timeline, id = datavar).items(30):
            try:
                counter= counter+1
                Total_Data.append(status)
                time.sleep()
            except Exception as e:
                pass
except Exception as e:
    pass
print(len(Total_Data))

210


In [50]:
import urllib.parse
import pandas as pd

def process_http(string):
    url_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if s and n:
            url_count += 1
    return url_count

def process_hashtag(string):
    hashtag_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '#':
            hashtag_count += 1
    return hashtag_count

def process_mention(string):
    mention_count=0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '@':
            mention_count += 1
    return mention_count
def process_data(Total_Data):
    TwittID = [tweet.id for tweet in Total_Data]
    # Making the dataset in pandas frame
    Data = pd.DataFrame(TwittID, columns = ['TwittID'])
    # processing the data in Tweet level
    
    Data["TextData"] = [tweet.text for tweet in Total_Data]
    Data["TweetCreatedAt"] = [tweet.created_at for tweet in Total_Data]
    Data["RetweetCount"] = [tweet.retweet_count for tweet in Total_Data]
    Data["TweetFavouriteCount"] = [tweet.favorite_count for tweet in Total_Data]
    Data["TweetSource"] = [tweet.source for tweet in Total_Data]
    
    # processing the data in User Graph level
    
    Data["UserID"] = [tweet.author.id for tweet in Total_Data]
    Data["UserScreenName"] = [tweet.author.screen_name for tweet in Total_Data]
    Data["UserName"] = [tweet.author.name for tweet in Total_Data]
    Data["UserCreatedAt"] = [tweet.author.created_at for tweet in Total_Data]
    Data["UserDescription"] = [tweet.author.description for tweet in Total_Data]
    Data["UserDescriptionLength"] = [len(tweet.author.description) for tweet in Total_Data]
    Data["UserFollowersCount"] = [tweet.author.followers_count for tweet in Total_Data]
    Data["UserFriendsCount"] = [tweet.author.friends_count for tweet in Total_Data]
    Data["UserLocation"] = [tweet.author.location for tweet in Total_Data]
    
    # Data["url"] = [tweet.author.url for in Total_Data]
    # Data["User_mention"] = [user_mentions.author.screen_name for tweet in Total_Data]
    # Data["HashTag"] = [hashtag.text for tweet in Total_Data]
    
    Data["HttpCount"] = [process_http(tweet.text) for tweet in Total_Data]
    Data["HashtagCount"] = [process_hashtag(tweet.text) for tweet in Total_Data]
    Data["MentionCount"] = [process_mention(tweet.text) for tweet in Total_Data]
    Data["TweetCount"] = [tweet.author.statuses_count for tweet in Total_Data]
    return Data
Data = process_data(Total_Data)
Data.shape

(210, 19)

In [51]:
# Saving data with item space separating
Data.to_csv('SpamData.csv', sep=',' , encoding='utf8')

## Spam And Genuine Data Merge

In [115]:
import pandas as pd
Total_leg_data = pd.read_csv('RajiniFriends.csv')
Total_leg_data.fillna(0, inplace=True)
Total_leg_data.drop('Unnamed: 0',axis=1,inplace=True)
Total_leg_data.shape

(720, 19)

In [116]:
Total_leg_data

,TwittID,TextData,TweetCreatedAt,RetweetCount,TweetFavouriteCount,TweetSource,UserID,UserScreenName,UserName,UserCreatedAt,UserDescription,UserDescriptionLength,UserFollowersCount,UserFriendsCount,UserLocation,HttpCount,HashtagCount,MentionCount,TweetCount
0,1399625976800153600,RT @karthiksubbaraj: Here's #JagameThandhiramT...,2021-06-01 07:16:51,3227,0,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,1,2,4,12891
1,1399625927089217536,RT @dhanushkraja: What would have been a great...,2021-06-01 07:16:39,7527,0,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,0,0,1,12891
2,1395367947552522241,This track and it’s video are 🔥 \nAwesome work...,2021-05-20 13:16:58,302,3645,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,1,1,3,12891
3,1395030557092769794,#URGENT 🆘\nEmergency need of 72 vials of #Amph...,2021-05-19 14:56:17,925,2715,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,1,2,0,12891
4,1392771872056479748,RT @soundaryaarajni: Our Thalaivar gets his va...,2021-05-13 09:21:05,4565,0,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,0,2,1,12891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,1402144157053853703,RT @news18dotcom: @AmanKayamHai_ In the detail...,2021-06-08 06:03:12,2,0,TweetDeck,6509832,CNNnews18,News18,2007-06-01 20:31:01,"Lightning fast alerts, #BreakingNews from Indi...",62,4597047,403,India,0,0,3,881487
716,1402143957325258753,RT @news18dotcom: @AmanKayamHai_ News18's @Ama...,2021-06-08 06:02:24,3,0,TweetDeck,6509832,CNNnews18,News18,2007-06-01 20:31:01,"Lightning fast alerts, #BreakingNews from Indi...",62,4597047,403,India,0,0,3,881487
717,1402142378102640643,It is learnt that the Centre had planned to am...,2021-06-08 05:56:08,3,11,Twitter Web App,6509832,CNNnews18,News18,2007-06-01 20:31:01,"Lightning fast alerts, #BreakingNews from Indi...",62,4597047,403,India,1,0,0,881487
718,1402140713614299136,"It was propagated as Modi vaccine: @RajeevRai,...",2021-06-08 05:49:31,1,7,Grabyo,6509832,CNNnews18,News18,2007-06-01 20:31:01,"Lightning fast alerts, #BreakingNews from Indi...",62,4597047,403,India,1,0,1,881487


In [105]:
Total_leg_data[["UserFollowersCount"]]

,UserFollowersCount
0,8138319
1,8138319
2,8138319
3,8138319
4,8138319
...,...
715,4597047
716,4597047
717,4597047
718,4597047


In [117]:
Total_leg_data[['RetweetCount']] = Total_leg_data[['RetweetCount']].astype(float)
Total_leg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TwittID                720 non-null    int64  
 1   TextData               720 non-null    object 
 2   TweetCreatedAt         720 non-null    object 
 3   RetweetCount           720 non-null    float64
 4   TweetFavouriteCount    720 non-null    int64  
 5   TweetSource            720 non-null    object 
 6   UserID                 720 non-null    int64  
 7   UserScreenName         720 non-null    object 
 8   UserName               720 non-null    object 
 9   UserCreatedAt          720 non-null    object 
 10  UserDescription        720 non-null    object 
 11  UserDescriptionLength  720 non-null    int64  
 12  UserFollowersCount     720 non-null    int64  
 13  UserFriendsCount       720 non-null    int64  
 14  UserLocation           720 non-null    object 
 15  HttpCo

In [118]:
Total_leg_data.shape

(720, 19)

In [119]:
# to see how many people have zero tweet
Total_leg_data = Total_leg_data[Total_leg_data.TweetCount!=0]
len(Total_leg_data[Total_leg_data.TweetCount<30])

0

In [120]:
Total_leg_data[["RetweetCount"]] = Total_leg_data[["RetweetCount"]].astype(float)
Total_leg_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TwittID                720 non-null    int64  
 1   TextData               720 non-null    object 
 2   TweetCreatedAt         720 non-null    object 
 3   RetweetCount           720 non-null    float64
 4   TweetFavouriteCount    720 non-null    int64  
 5   TweetSource            720 non-null    object 
 6   UserID                 720 non-null    int64  
 7   UserScreenName         720 non-null    object 
 8   UserName               720 non-null    object 
 9   UserCreatedAt          720 non-null    object 
 10  UserDescription        720 non-null    object 
 11  UserDescriptionLength  720 non-null    int64  
 12  UserFollowersCount     720 non-null    int64  
 13  UserFriendsCount       720 non-null    int64  
 14  UserLocation           720 non-null    object 
 15  HttpCo

In [121]:
Total_leg_data.shape

(720, 19)

In [122]:
Total_leg_data.loc[:,"AvgHashtag"] = (Total_leg_data.groupby('UserID')["HashtagCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgURLCount"] = (Total_leg_data.groupby('UserID')["HttpCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgMention"] = (Total_leg_data.groupby('UserID')["MentionCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgRetweet"] = (Total_leg_data.groupby('UserID')["RetweetCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgFavCount"] = (Total_leg_data.groupby('UserID')["TweetFavouriteCount"].transform('sum'))/30

In [123]:
Total_leg_data.shape

(720, 24)

In [ ]:
# Add a Column to LEgitimate Data that this is not Spam =0
leg_data.loc[:, "SpammerOrNot"]=0
leg_data.sample(10)

In [124]:
Total_leg_data.loc[:, "SpammerOrNot"]=0

In [125]:
Total_leg_data.shape

(720, 25)

In [127]:
Total_leg_data.SpammerOrNot.value_counts()

0    720
Name: SpammerOrNot, dtype: int64

In [74]:
leg_data["TweetCount"].describe()

count        26.000000
mean     391523.692308
std      335314.638177
min        2775.000000
25%       29914.000000
50%      360549.000000
75%      673337.250000
max      900465.000000
Name: TweetCount, dtype: float64

In [128]:
# Now Loading Spammer Data 
Total_spam_data = pd.read_csv("SpamData.csv")
Total_spam_data.fillna(0, inplace=True)
Total_spam_data.drop('Unnamed: 0',axis=1,inplace=True)
Total_spam_data.shape

(210, 19)

In [129]:
Total_spam_data[['RetweetCount']] = Total_spam_data[['RetweetCount']].astype(float)
Total_spam_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TwittID                210 non-null    int64  
 1   TextData               210 non-null    object 
 2   TweetCreatedAt         210 non-null    object 
 3   RetweetCount           210 non-null    float64
 4   TweetFavouriteCount    210 non-null    int64  
 5   TweetSource            210 non-null    object 
 6   UserID                 210 non-null    int64  
 7   UserScreenName         210 non-null    object 
 8   UserName               210 non-null    object 
 9   UserCreatedAt          210 non-null    object 
 10  UserDescription        210 non-null    object 
 11  UserDescriptionLength  210 non-null    int64  
 12  UserFollowersCount     210 non-null    int64  
 13  UserFriendsCount       210 non-null    int64  
 14  UserLocation           210 non-null    object 
 15  HttpCo

In [130]:
Total_spam_data = Total_spam_data[Total_spam_data.TweetCount!=0]
len(Total_spam_data[Total_spam_data.TweetCount<30])

0

In [131]:
Total_spam_data.loc[:,'AvgHashtag'] = (Total_spam_data.groupby('UserID')["HashtagCount"].transform('sum'))/30
Total_spam_data.loc[:,'AvgURLCount'] = (Total_spam_data.groupby('UserID')["HttpCount"].transform('sum'))/30
Total_spam_data.loc[:,'AvgMention'] = (Total_spam_data.groupby('UserID')["MentionCount"].transform('sum'))/30
Total_spam_data.loc[:,'AvgRetweet'] = (Total_spam_data.groupby('UserID')["RetweetCount"].transform('sum'))/30
Total_spam_data.loc[:,'AvgFavCount'] = (Total_spam_data.groupby('UserID')["TweetFavouriteCount"].transform('sum'))/30

In [133]:
Total_spam_data.loc[:, "SpammerOrNot"]=1

In [134]:
Total_spam_data.shape

(210, 25)

In [135]:
frames = [Total_leg_data, Total_spam_data]
Total_data = pd.concat(frames, axis=0, sort=False)
Total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 930 entries, 0 to 209
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TwittID                930 non-null    int64  
 1   TextData               930 non-null    object 
 2   TweetCreatedAt         930 non-null    object 
 3   RetweetCount           930 non-null    float64
 4   TweetFavouriteCount    930 non-null    int64  
 5   TweetSource            930 non-null    object 
 6   UserID                 930 non-null    int64  
 7   UserScreenName         930 non-null    object 
 8   UserName               930 non-null    object 
 9   UserCreatedAt          930 non-null    object 
 10  UserDescription        930 non-null    object 
 11  UserDescriptionLength  930 non-null    int64  
 12  UserFollowersCount     930 non-null    int64  
 13  UserFriendsCount       930 non-null    int64  
 14  UserLocation           930 non-null    object 
 15  HttpCo

In [136]:
Total_data.reset_index()
Total_data.to_csv('Total_data.csv', sep=',', encoding='utf8')

In [137]:
df = pd.read_csv('Total_data.csv')

In [140]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [141]:
df.shape

(930, 25)

In [144]:
df.head(1)

,TwittID,TextData,TweetCreatedAt,RetweetCount,TweetFavouriteCount,TweetSource,UserID,UserScreenName,UserName,UserCreatedAt,UserDescription,UserDescriptionLength,UserFollowersCount,UserFriendsCount,UserLocation,HttpCount,HashtagCount,MentionCount,TweetCount,AvgHashtag,AvgURLCount,AvgMention,AvgRetweet,AvgFavCount,SpammerOrNot
0,1399625976800153600,RT @karthiksubbaraj: Here's #JagameThandhiramT...,2021-06-01 07:16:51,3227.0,0,Twitter for iPhone,216447259,anirudhofficial,Anirudh Ravichander,2010-11-16 19:13:06,Music Composer/Singer in the Indian Film Industry,49,8138319,207,Chennai,1,2,4,12891,1.133333,0.433333,1.566667,2806.4,6632.266667,0


In [143]:
df.shape

(930, 25)

In [145]:
df.columns

Index(['TwittID', 'TextData', 'TweetCreatedAt', 'RetweetCount',
       'TweetFavouriteCount', 'TweetSource', 'UserID', 'UserScreenName',
       'UserName', 'UserCreatedAt', 'UserDescription', 'UserDescriptionLength',
       'UserFollowersCount', 'UserFriendsCount', 'UserLocation', 'HttpCount',
       'HashtagCount', 'MentionCount', 'TweetCount', 'AvgHashtag',
       'AvgURLCount', 'AvgMention', 'AvgRetweet', 'AvgFavCount',
       'SpammerOrNot'],
      dtype='object')

In [146]:
Lis = ['RetweetCount','TweetFavouriteCount','UserDescriptionLength','UserFollowersCount', 'UserFriendsCount','HttpCount','HashtagCount', 'MentionCount', 'TweetCount', 'AvgHashtag','AvgURLCount', 'AvgMention', 'AvgRetweet', 'AvgFavCount']

In [147]:
X = df[Lis]

In [148]:
X

,RetweetCount,TweetFavouriteCount,UserDescriptionLength,UserFollowersCount,UserFriendsCount,HttpCount,HashtagCount,MentionCount,TweetCount,AvgHashtag,AvgURLCount,AvgMention,AvgRetweet,AvgFavCount
0,3227.0,0,49,8138319,207,1,2,4,12891,1.133333,0.433333,1.566667,2806.4,6632.266667
1,7527.0,0,49,8138319,207,0,0,1,12891,1.133333,0.433333,1.566667,2806.4,6632.266667
2,302.0,3645,49,8138319,207,1,1,3,12891,1.133333,0.433333,1.566667,2806.4,6632.266667
3,925.0,2715,49,8138319,207,1,2,0,12891,1.133333,0.433333,1.566667,2806.4,6632.266667
4,4565.0,0,49,8138319,207,0,2,1,12891,1.133333,0.433333,1.566667,2806.4,6632.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,0.0,0,98,275,922,1,0,3,8481,0.166667,0.600000,1.133333,65.9,0.400000
926,0.0,5,98,275,922,1,0,1,8481,0.166667,0.600000,1.133333,65.9,0.400000
927,25.0,0,98,275,922,1,0,1,8481,0.166667,0.600000,1.133333,65.9,0.400000
928,0.0,3,98,275,922,0,0,3,8481,0.166667,0.600000,1.133333,65.9,0.400000


In [149]:
y = df['SpammerOrNot']

In [150]:
y

0      0
1      0
2      0
3      0
4      0
      ..
925    1
926    1
927    1
928    1
929    1
Name: SpammerOrNot, Length: 930, dtype: int64

In [152]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.20, random_state=24)

In [158]:
from sklearn.linear_model import LogisticRegression


In [159]:
model=LogisticRegression(max_iter=1000)

In [160]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [161]:
prediction=model.predict(X_test)

In [162]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [164]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,plot_roc_curve


In [166]:
prediction

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [170]:
Acc = accuracy_score(y_test, prediction)

In [173]:
print("LR Model Acc : ",Acc*100)

LR Model Acc :  95.33


In [174]:
print('Name of the Friends of user')
friends = []
for friend in tweepy.Cursor(api.friends, screen_name = 'RajiniFollowers').items(20):
    try:
                friends.append(friend.screen_name)
                print(friend.screen_name)
                time.sleep()
    except Exception as e:
                pass

with open("TestOrg.txt", "w") as f:
     for item in friends:
        f.write("%s\n" % item)

Name of the Friends of user
Narendra4_U
Yuvrajganesan
latharajnikanth
immancomposer
directorsiva
sunpictures
anirudhofficial
rajinikanth
soundaryaarajni
dhanushkraja


In [175]:
Total_Data = []
fo = open("TestOrg.txt", "r")
f = fo.readlines()
fo.close()
dataset = map(lambda s: s.strip(),f)
try:
    for datavar in dataset:
        data = api.get_user(datavar)
        counter = 0
        for status in tweepy.Cursor(api.user_timeline, id = datavar).items(30):
            try:
                counter= counter+1
                Total_Data.append(status)
                time.sleep()
            except Exception as e:
                pass
except Exception as e:
    pass
print(len(Total_Data))

300


In [177]:
import urllib.parse
import pandas as pd

def process_http(string):
    url_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if s and n:
            url_count += 1
    return url_count

def process_hashtag(string):
    hashtag_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '#':
            hashtag_count += 1
    return hashtag_count

def process_mention(string):
    mention_count=0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '@':
            mention_count += 1
    return mention_count
def process_data(Total_Data):
    TwittID = [tweet.id for tweet in Total_Data]
    # Making the dataset in pandas frame
    Data = pd.DataFrame(TwittID, columns = ['TwittID'])
    # processing the data in Tweet level
    
    Data["TextData"] = [tweet.text for tweet in Total_Data]
    Data["TweetCreatedAt"] = [tweet.created_at for tweet in Total_Data]
    Data["RetweetCount"] = [tweet.retweet_count for tweet in Total_Data]
    Data["TweetFavouriteCount"] = [tweet.favorite_count for tweet in Total_Data]
    Data["TweetSource"] = [tweet.source for tweet in Total_Data]
    
    # processing the data in User Graph level
    
    Data["UserID"] = [tweet.author.id for tweet in Total_Data]
    Data["UserScreenName"] = [tweet.author.screen_name for tweet in Total_Data]
    Data["UserName"] = [tweet.author.name for tweet in Total_Data]
    Data["UserCreatedAt"] = [tweet.author.created_at for tweet in Total_Data]
    Data["UserDescription"] = [tweet.author.description for tweet in Total_Data]
    Data["UserDescriptionLength"] = [len(tweet.author.description) for tweet in Total_Data]
    Data["UserFollowersCount"] = [tweet.author.followers_count for tweet in Total_Data]
    Data["UserFriendsCount"] = [tweet.author.friends_count for tweet in Total_Data]
    Data["UserLocation"] = [tweet.author.location for tweet in Total_Data]
    
    # Data["url"] = [tweet.author.url for in Total_Data]
    # Data["User_mention"] = [user_mentions.author.screen_name for tweet in Total_Data]
    # Data["HashTag"] = [hashtag.text for tweet in Total_Data]
    
    Data["HttpCount"] = [process_http(tweet.text) for tweet in Total_Data]
    Data["HashtagCount"] = [process_hashtag(tweet.text) for tweet in Total_Data]
    Data["MentionCount"] = [process_mention(tweet.text) for tweet in Total_Data]
    Data["TweetCount"] = [tweet.author.statuses_count for tweet in Total_Data]
    return Data
Data = process_data(Total_Data)
Data.shape

(300, 19)